In [23]:
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv


load_dotenv()

with open(
    r'C:\course_video\llm_basic\llm_0527\남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt', 'r', encoding='utf-8'
    ) as file:
        script = file.read()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
        '''
        사용자가 제공한 문서를 참고하여, 사용자 질문에 답변합니다. 
        답변은 문서 내용을 기반으로 하고, 
        해당 페이지도 표시합니다.
        참고문서: "{book}"
        '''),
        ("placeholder", "{chat_history}"), 
        ("human", "{input}"), 
    ]
)

chain = prompt | ChatOpenAI(model='gpt-4.1-nano', max_completion_tokens=50,) | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory( 
    chain,
    get_session_history, 
    input_messages_key= 'input',
    history_messages_key= 'chat_history'
    )

In [24]:
request = input('질문하세요 >>')

wrapped_chain.invoke(
    {'book' : script, 'input' : request},
    config={'configurable': {'session_id': 123456}},
)

'육아휴직 조건은 다음과 같습니다:\n- 임신 중인 여성 근로자가 모성 보호 또는 만 8세 이하 또는 초등학교 2학년 이하 자녀를 양육하기 위해 휴직을 신청하는 경우'